# Data Cleaning Notebook

This notebook provides a systematic approach to cleaning your data based on findings from the exploration phase.

## Table of Contents
1. [Setup and Data Loading](#setup-and-data-loading)
2. [Missing Values Treatment](#missing-values-treatment)
3. [Outlier Detection and Treatment](#outlier-detection-and-treatment)
4. [Duplicate Detection](#duplicate-detection)
5. [Data Type Corrections](#data-type-corrections)
6. [Data Validation](#data-validation)
7. [Export Cleaned Data](#export-cleaned-data)

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# EDA toolkit imports
import sys
sys.path.append('../src')
sys.path.append('../plots')

from data_loader import DataLoader
from data_cleaner import DataCleaner
from statistical_analysis import StatisticalAnalyzer
from box_plot import create_outlier_analysis_plot
from utils import PlotConfig

# Configure plotting
plot_config = PlotConfig()
plot_config.set_style()

print("Setup complete!")

## Setup and Data Loading

In [ ]:
# Load your data (replace with your actual data path)
loader = DataLoader()

# For demonstration, create sample data with issues
np.random.seed(42)
sample_data = {
    'ID': range(1, 1001),
    'Age': np.random.randint(18, 80, 1000),
    'Income': np.random.normal(50000, 15000, 1000),
    'Education Level': np.random.choice(['High School', 'Bachelor', 'Master', 'PhD', 'bachelor', 'HIGH SCHOOL'], 1000),
    'Experience_Years': np.random.randint(0, 40, 1000),
    'Salary': np.random.normal(60000, 20000, 1000),
    'Category': np.random.choice(['A', 'B', 'C', 'a'], 1000)
}

# Add outliers and missing values
sample_data['Income'][np.random.choice(1000, 50, replace=False)] = np.nan
sample_data['Salary'][np.random.choice(1000, 30, replace=False)] = np.nan
sample_data['Income'][np.random.choice(1000, 10, replace=False)] = np.random.uniform(200000, 500000, 10)
sample_data['Age'][np.random.choice(1000, 5, replace=False)] = np.random.choice([150, 200], 5)

# Add duplicates
data = pd.DataFrame(sample_data)
duplicate_rows = data.sample(20).copy()
data = pd.concat([data, duplicate_rows], ignore_index=True)

print(f"Original data shape: {data.shape}")
print(f"Data loaded with intentional issues for cleaning demonstration")

# Initialize cleaner
cleaner = DataCleaner()

## Missing Values Treatment

In [ ]:
# Analyze missing value patterns
missing_patterns = cleaner.detect_missing_patterns(data)

print("Missing Values Analysis:")
display(missing_patterns['by_column'])

print(f"\nRows with any missing values: {missing_patterns['rows_with_missing']} ({missing_patterns['rows_with_missing_percentage']:.2f}%)")
print(f"Columns with no missing values: {missing_patterns['complete_columns']}")
print(f"Highly missing columns (>50%): {missing_patterns['highly_missing_columns']}")

In [ ]:
# Define missing value treatment strategy
missing_strategy = {
    'Income': 'median',  # Use median for income
    'Salary': 'median',  # Use median for salary
    'Education Level': 'mode',  # Use mode for categorical
    'Category': 'mode'
}

# Handle missing values
data_cleaned = cleaner.handle_missing_values(data, strategy=missing_strategy, threshold=0.5)

print(f"After missing value treatment:")
print(f"Shape: {data_cleaned.shape}")
print(f"Missing values remaining: {data_cleaned.isnull().sum().sum()}")

## Outlier Detection and Treatment

In [ ]:
# Detect outliers in numeric columns
numeric_cols = data_cleaned.select_dtypes(include=[np.number]).columns.tolist()
outlier_analysis = cleaner.detect_outliers(data_cleaned, columns=numeric_cols, method='iqr')

print("Outlier Analysis (IQR method):")
for col, info in outlier_analysis.items():
    if info['count'] > 0:
        print(f"\n{col}:")
        print(f"  Outliers found: {info['count']} ({info['percentage']:.2f}%)")
        print(f"  Outlier values: {info['values'][:5]}{'...' if len(info['values']) > 5 else ''}")

In [ ]:
# Visualize outliers for key variables
for col in ['Age', 'Income', 'Salary']:
    if col in data_cleaned.columns:
        print(f"\nOutlier Analysis for {col}:")
        create_outlier_analysis_plot(data_cleaned, col, 
                                   save_path=f'../figures/temp/{col}_outlier_analysis.png')
        plt.show()

In [ ]:
# Handle outliers (choose appropriate strategy)
# Options: 'remove', 'cap', 'transform'
outlier_strategy = 'cap'  # Cap outliers to reasonable bounds

data_cleaned = cleaner.handle_outliers(data_cleaned, outlier_analysis, strategy=outlier_strategy)

print(f"After outlier treatment ({outlier_strategy}):")
print(f"Shape: {data_cleaned.shape}")

# Re-check outliers
new_outlier_analysis = cleaner.detect_outliers(data_cleaned, columns=numeric_cols, method='iqr')
for col, info in new_outlier_analysis.items():
    print(f"{col}: {info['count']} outliers remaining")

## Duplicate Detection

In [ ]:
# Detect duplicates
duplicate_info = cleaner.detect_duplicates(data_cleaned)

print(f"Duplicate Analysis:")
print(f"Duplicate rows found: {duplicate_info['count']} ({duplicate_info['percentage']:.2f}%)")

if duplicate_info['count'] > 0:
    print(f"\nFirst few duplicate rows:")
    display(data_cleaned[data_cleaned.duplicated()].head())
    
    # Remove duplicates
    data_cleaned = cleaner.remove_duplicates(data_cleaned, keep='first')
    print(f"\nAfter removing duplicates:")
    print(f"Shape: {data_cleaned.shape}")
else:
    print("No duplicates found.")

## Data Type Corrections

In [ ]:
# Standardize column names
data_cleaned = cleaner.standardize_column_names(data_cleaned)

print("Column names standardized:")
print(data_cleaned.columns.tolist())

# Check data types
print("\nData types:")
print(data_cleaned.dtypes)

# Fix categorical data inconsistencies
if 'education_level' in data_cleaned.columns:
    print("\nEducation level values before cleaning:")
    print(data_cleaned['education_level'].value_counts())
    
    # Standardize education levels
    education_mapping = {
        'high school': 'High School',
        'HIGH SCHOOL': 'High School',
        'bachelor': 'Bachelor',
        'Bachelor': 'Bachelor',
        'Master': 'Master',
        'PhD': 'PhD'
    }
    
    data_cleaned['education_level'] = data_cleaned['education_level'].map(education_mapping).fillna(data_cleaned['education_level'])
    
    print("\nEducation level values after cleaning:")
    print(data_cleaned['education_level'].value_counts())

# Standardize category values
if 'category' in data_cleaned.columns:
    print("\nCategory values before cleaning:")
    print(data_cleaned['category'].value_counts())
    
    data_cleaned['category'] = data_cleaned['category'].str.upper()
    
    print("\nCategory values after cleaning:")
    print(data_cleaned['category'].value_counts())

## Data Validation

In [ ]:
# Validate cleaned data
print("Data Validation Results:")
print("=" * 40)

# Check for remaining issues
print(f"1. Shape: {data_cleaned.shape}")
print(f"2. Missing values: {data_cleaned.isnull().sum().sum()}")
print(f"3. Duplicates: {data_cleaned.duplicated().sum()}")

# Validate numeric ranges
numeric_cols = data_cleaned.select_dtypes(include=[np.number]).columns
print(f"\n4. Numeric column ranges:")
for col in numeric_cols:
    min_val = data_cleaned[col].min()
    max_val = data_cleaned[col].max()
    print(f"   {col}: {min_val:.2f} to {max_val:.2f}")

# Validate categorical consistency
categorical_cols = data_cleaned.select_dtypes(include=['object', 'category']).columns
print(f"\n5. Categorical column unique values:")
for col in categorical_cols:
    unique_count = data_cleaned[col].nunique()
    print(f"   {col}: {unique_count} unique values")

# Generate cleaning report
cleaning_report = cleaner.get_cleaning_report()
print(f"\n6. Cleaning Summary:")
for step, details in cleaning_report.items():
    print(f"   {step}: {details}")

## Export Cleaned Data

In [ ]:
# Export cleaned data
output_path = '../data/processed/cleaned_data.csv'
data_cleaned.to_csv(output_path, index=False)

print(f"Cleaned data exported to: {output_path}")
print(f"Final shape: {data_cleaned.shape}")

# Create before/after comparison
comparison = pd.DataFrame({
    'Metric': ['Rows', 'Columns', 'Missing Values', 'Duplicates'],
    'Before': [data.shape[0], data.shape[1], data.isnull().sum().sum(), data.duplicated().sum()],
    'After': [data_cleaned.shape[0], data_cleaned.shape[1], 
             data_cleaned.isnull().sum().sum(), data_cleaned.duplicated().sum()]
})

print("\nBefore vs After Comparison:")
display(comparison)

print("\nData cleaning completed successfully!")
print("Ready for feature analysis and modeling.")